### **Training** ###
This notebook trains the YOLOv8 segmentation model to detect a new rooftop class. We considered the available YOLO configurations—nano, small, medium, large, and xl. While the larger sizes generally yield better results, they come with higher computational demands. To balance performance and computational expense, we chose the small YOLO model.

The training datasets were derived from the RAMP dataset and prepared using the preprocessing notebook. To optimize the model results, we trained YOLO using data version 1, 2, and 3. Each data version was successively pruned (V3, V2, .. V0), meaning V3 was pruned from V2 and so on to V0 so that each version contains the prunings from all of the previous.

RAMP-DATA-V0: 'Full Dataset' is 100k image-label pairs converted from RAMP's TIF-GEOJSON formats within 22 regional folders and stored in train-val-test folders with a 70-15-15 split; zipped at 15.2 GB downloaded from Source Cooperative.

YOLO-DATA-V1: 'Preprocessing' was a process that resulted in 83k image-label pairs in JPG-TXT formats from 20 regions, zipped at 3.5 GB. We pruned 17k images from V0, reducing the background images from 18% to 5%. Shanghai and Paris regions were removed due to image format issues. A minimal manual review discovered a few labeling issues (n=38), ensuring the highest data quality.

YOLO-DATA-V2: 'Pruning #2' resulted in 79k image-label pairs in JPG-TXT formats from 20 regions, zipped at 3.3 GB. We scored and analyzed inference results from the YOLO model trained on DATA-V1. The analysis revealed double the expected number of left-tail outliers. To improve labeling accuracy, we removed 4k outliers, assuming that most of these labels were errors.

YOLO-DATA-V3: 'Zoom 19' resulted in 55k image-label pairs in JPG-TXT formats from 20 regions, zipped at 2.4 GB. We extracted the x and y scales from the TIF files and found the range to be between 0.3 and 0.52 meters per pixel, with 70% between 0.3 and 0.4. To test the effect of training near Zoom 19, which equates to 0.3 meters per pixel, we removed all image labels with scales greater than 0.4.

To relieve the time constraints, we recommend training with a GPU. Google Colab's High-RAM T4 can train 100 epochs in 15 hours using a large batch size and cache enabled to minimize disk I/O. Using a 2023 laptop with a CPU, training YOLO for 100 epochs takes about 2.4 weeks.

In [1]:
!pip3 install -q ultralytics

In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu116

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu116


In [3]:
import torch

#Use GPU is avalaibe
if torch.cuda.is_available():
  print("CUDA is available. Using GPU for training.")
  device = torch.device("cuda")
else:
  print("CUDA is not available. Using CPU for training.")
  device = torch.device("cpu")



CUDA is available. Using GPU for training.


In [4]:
from ultralytics import YOLO
import os
import yaml

In [5]:
for entry in os.scandir('.'):
    if entry.is_file():
        print(entry.name,entry.stat().st_size)

.python_history 45
yolov8m-seg.pt 54921020
yolov8x-seg.pt 144101612
.sudo_as_admin_successful 0
data3_ramp_data_yolo_iou_scale.zip 2419400988
yolov8n-seg.pt 7071756
data1_ramp_data_yolo.zip 3508121128
.bash_history 6154
yolov8s-seg.pt 23914764
.motd_shown 0
cuda-keyring_1.1-1_all.deb 4328
args_500e.yaml 504
yolo11n.pt 5613764
.profile 807
yolov8l-seg.pt 92417004
.bash_logout 220
.rediscli_history 12
args_500e_tuned.yaml 516
Copy_of_training.ipynb 15697
.bashrc 3771
yolov8_-seg.zip 298131335
data.yaml 99


In [7]:
data_choice = input("Which dataset do you want to download? Enter 'v1', 'v2', or 'v3': ")

if data_choice.lower() == 'v1':
    print("\nunzipping data1_ramp_data_yolo.zip")
    # !unzip -q -o data3*.zip
elif data_choice.lower() == 'v2':
    print("Invalid input.")
elif data_choice.lower() == 'v3':
    print("\nunzipping data3_ramp_data_yolo.zip")
    # !unzip -q -o data3*.zip
else:
    print("Invalid input.")

print('Unzipping models...')
# !unzip -q -o yolov8_-seg.zip

# # Download and unzip the specified data version
# if data_choice.lower() == 'v1':
#     print("\nDownloading and unzipping v1...")
#     !gdown --fuzzy https://drive.google.com/file/d/1iosxGOyBXNXoAdvdxIR7DTSWKRVLijqa/view?usp=sharing
#     !unzip -q data1*.zip
# elif data_choice.lower() == 'v2':
#     print("\nDownloading and unzipping v2...")
#     !gdown --fuzzy https://drive.google.com/file/d/1zXwvKLxzzOBp1bQ67nqfOgM55BI6sHho/view?usp=sharing
#     !unzip -q data2*.zip
# elif data_choice.lower() == 'v3':
#     print("\nDownloading and unzipping v3...")
#     !gdown --fuzzy https://drive.google.com/file/d/1K-jaOylXjukvCC6Uu8Ms2e7LxfL0-qWg/view?usp=sharing
#     !unzip -q data3*.zip
# else:
#     print("Invalid input.")

# Download and unzip models if not found.
# if not os.path.exists('yolov8s-seg.pt'):
#     print('\nDownloading and unzipping YOLOv8 models...')
#     !gdown --fuzzy https://drive.google.com/file/d/189U5fTomw3-QPjPpkppiXVSKa-sVIQIl/view?usp=sharing
#     print('Unzipping models...')
#     !unzip -q yolov8_-seg.zip
#     print('Models downloaded')
# else:
#     print('\nModels already downloaded')

# Download the training arguments
# print('\nDownloading training arguments...')
# !gdown --fuzzy https://drive.google.com/file/d/13KLv4ywEVNVQHWhahIapSDDEzJWcAmT_/view?usp=sharing
# !gdown --fuzzy https://drive.google.com/file/d/1_N8H6akiKqFwezDjqfjVuXDvuP6zJZGP/view?usp=sharing

Which dataset do you want to download? Enter 'v1', 'v2', or 'v3':  v3



unzipping data3_ramp_data_yolo.zip
Unzipping models...


In [8]:
# Write the YAML file for YOLO that contains the class names and train/val paths
YAML = 'data.yaml'

# Set the path based on the version of the dataset
if data_choice == 'v1':
    path = f'{os.getcwd()}/ramp_data_yolo'
elif data_choice == 'v2':
    path = f'{os.getcwd()}/ramp_data_yolo_iou'
elif data_choice == 'v3':
    path = f'{os.getcwd()}/ramp_data_yolo_iou_scale'

# Set the attribute for the YAML file
attr = {
    'path': path,
    'train': 'train/images',
    'val': 'val/images',
    'names': {
        0: 'rooftop'
    }
}

# Print the path
print(f'Writing the data file with path={path}')

# Write the file
with open(YAML, 'w') as f:
    yaml.dump(attr, f)

Writing the data file with path=/home/omran/ramp_data_yolo_iou_scale


In [9]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [10]:
# # HIGHLY RECOMMENDED if your training is of a long duration
# # Connect to Google Drive so that if session is terminated you can resume
# from google.colab import drive

# # Uncomment if needed
# drive.mount('/content/drive')

In [12]:
# For testing purposes the fraction argument can be used to train on a subset of the data
FRACTION = 1.0

# Setting the starting model
MODEL = 'yolov8s-seg.pt'

# Setting the project name
PROJECT = '/'

# Load the arguments from YAML
ARGS_FN = 'args_500e_tuned.yaml'
with open(ARGS_FN, 'r') as f:
    args = yaml.safe_load(f)

# Generate a name for the training session
name = f'{os.getcwd()}/{MODEL}_data_{data_choice}'
name = name.replace('.pt', '')

# Prompt the user for input
choice = input("Do you want to start or resume training? Enter 'new' for new training or 'resume' to resume training: ")

# Process the user input
if choice.lower() == 'new':
    print("\nStarting new training...\n")

    # Load the model
    model = YOLO(MODEL)

    # Train the model
    results = model.train(data=YAML, fraction=FRACTION, project=PROJECT, name=name,device=device, **args)

elif choice.lower() == 'resume':
    print("Resuming training...")

    # Load the model
    modelwpath = f'{PROJECT}/{name}/weights/last.pt'
    print(f'Loading model from {modelwpath}')
    model = YOLO(modelwpath)

    # Resume training
    results = model.train(resume=True)

else:
    print("Invalid input.")

Do you want to start or resume training? Enter 'new' for new training or 'resume' to resume training:  resume


Resuming training...
Loading model from ///home/omran/yolov8s-seg_data_v3/weights/last.pt
Ultralytics 8.3.24 🚀 Python-3.10.6 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine/trainer: task=segment, mode=train, model=/home/omran/yolov8s-seg_data_v3/weights/last.pt, data=data.yaml, epochs=500, time=None, patience=500, batch=64, imgsz=256, save=True, save_period=-1, cache=disk, device=cuda, workers=8, project=/, name=yolov8s-seg_data_v34, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=/home/omran/yolov8s-seg_data_v3/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=False, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, a

train: Scanning /home/omran/ramp_data_yolo_iou_scale/train/labels.cache... 38662 images, 2211 backgrounds, 0 corrupt: 10
train: Caching images (7.1GB Disk): 100%|██████████| 38662/38662 [00:02<00:00, 14265.39it/s]
val: Scanning /home/omran/ramp_data_yolo_iou_scale/val/labels.cache... 8363 images, 480 backgrounds, 0 corrupt: 100%|███
val: Caching images (1.5GB Disk): 100%|██████████| 8363/8363 [00:00<00:00, 14970.05it/s]


Plotting labels to /home/omran/yolov8s-seg_data_v34/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.00854' and 'momentum=0.95275' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.00058), 76 bias(decay=0.0)
Resuming training /home/omran/yolov8s-seg_data_v3/weights/last.pt from epoch 3 to 500 total epochs
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to /home/omran/yolov8s-seg_data_v34
Starting training for 500 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      3/500      3.94G      1.294      1.996      1.469      1.072         62        256: 100%|██████████| 605/605 [07:0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP5


...

                   all       8363     117177      0.762      0.685      0.743       0.48      0.761      0.673      0.734      0.428

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


    500/500      3.94G     0.8719      1.215      0.729     0.9046         83        256: 100%|██████████| 605/605 [05:3
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP5


                   all       8363     117177      0.761      0.686      0.743       0.48       0.76      0.673      0.734      0.428

498 epochs completed in 64.826 hours.
Optimizer stripped from /home/omran/yolov8s-seg_data_v34/weights/last.pt, 23.9MB
Optimizer stripped from /home/omran/yolov8s-seg_data_v34/weights/best.pt, 23.9MB

Validating /home/omran/yolov8s-seg_data_v34/weights/best.pt...
Ultralytics 8.3.24 🚀 Python-3.10.6 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLOv8s-seg summary (fused): 195 layers, 11,779,987 parameters, 0 gradients, 42.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP5


                   all       8363     117177      0.779      0.704      0.789      0.528      0.776      0.692      0.778      0.467
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /home/omran/yolov8s-seg_data_v34
